## 整数转化为任意进制

- 基本结束条件
- 规模变小

In [1]:
def toStr(n,base):
    converString='0123456789ABCDEF'
    if n<base:
        return converString[n]
    else:
        return toStr(n//base,base)+converString[n%base]

In [2]:
print(toStr(1453,16))

5AD


有些时候因为算法效率低和无限递归导致递归系统调用栈有限，但可以通过python自带的函数进行调整递归栈的规模

In [3]:
import sys

sys.getrecursionlimit()#查看递归栈的限制

3000

In [4]:
sys.setrecursionlimit(1000)
sys.getrecursionlimit()

1000

## 递归可视化

使用python内置的海龟画图实现递归调用的可视化

- 爬行：forward(n):backward(n)
- 转向：left(a);right(a)
- 抬笔放笔：penup();pendown()
- 笔属性：pensize(s):pencolor(c)

In [5]:
import turtle

t=turtle.Turtle()
for i in range(4):
    t.forward(100)
    t.left(90)
turtle.done()

那么就来画一棵二叉树吧

In [1]:
import turtle

t=turtle.Turtle()
t.left(90)
t.penup()
t.backward(100)
t.pendown()

def drawTree(t,lineLen):
    if lineLen>5:
        t.forward(lineLen)
        t.left(20)
        drawTree(t,lineLen-15)
        t.right(40)
        drawTree(t,lineLen-15)
        t.left(20)
        t.backward(lineLen)

drawTree(t,75)
turtle.done()


## 分治策略

运用贪心算法，先用大面额的钱币进行找零，逐步减小规模，对于一般常规货币体系比较适用。但此算法对于特殊面额货币（21元）找零问题得到的不是货币数量最少的最优解

In [22]:
def greedy(coinValueList,change):
    coinValueList.sort(reverse=True)
    amount=[]
    for coin in coinValueList:
        if change>=coin:
            amount.append(change//coin)
            change=change%coin
    return amount

In [26]:
greedy([1,5,10,21,25],63)

[2, 1, 3]

## 递归方法解决硬币兑换找零问题

$$
numCoins=\min\begin{cases}
1+numCoins(originalamount-1)\\
1+numCoins(originalamount-5)\\
1+numCoins(originalamount-10)\\
1+numCoins(originalamount-25)
\end{cases}
$$

使用不同面额进行兑换，找到兑换数量最少的方案

In [24]:
def recMC(coinValueList,change):
    minCoins=change#最小数量的硬币
    if change in coinValueList:
        return 1#基本结束条件
    else:
        for i in [c for c in coinValueList if c<= change]:
            numCoins=1+recMC(coinValueList,change-i)
            if numCoins<minCoins:
                minCoins=numCoins
    return minCoins

In [27]:
print(recMC([1,5,10,21,25],63))

3


In [28]:
def recDC(coinValueList,change,knownResults):
    minCoins=change
    if change in coinValueList:
        knownResults[change]=1
        return 1
    elif knownResults[change]>0:
        return knownResults[change]
    else:
        for i in [c for c in coinValueList if c<= change]:
            numCoins=1+recDC(coinValueList,change-i,knownResults)
            if numCoins<minCoins:
                minCoins=numCoins
                knownResults[change]=minCoins#保存找零change钱所需要的最小次数
    return minCoins

In [29]:
print(recDC([1,5,10,21,25],63,[0]*64))

3


## 找零兑换问题的动态规划解法

In [36]:
def dpMakeChange(coinValueList,change,minCoins):
    for cents in range(1,change+1):
        coinCount=cents#初始化一个最大值
        for j in [c for c in coinValueList if cents>=c]:
            if minCoins[cents-j]+1<coinCount:
                coinCount=minCoins[cents-j]+1
        minCoins[cents]=coinCount
    return minCoins[change]

In [37]:
dpMakeChange([1,5,10,21,25],63,[0]*64)

3

In [48]:
def dpMakeChange1(coinValueList,change,minCoins,coinsUsed):
    for cents in range(change+1):
        coinCount=cents
        newCoin=1
        for j in [c for c in coinValueList if c<=cents]:
            if minCoins[cents-j]+1<coinCount:
                coinCount=minCoins[cents-j]+1
                newCoin=j
        minCoins[cents]=coinCount
        coinsUsed[cents]=newCoin
    return minCoins[change],coinsUsed

In [49]:
def printCoins(coinUsed,change):
    coin=change
    while coin>0:
        thisCoin=coinUsed[coin]
        print(thisCoin)
        coin=coin-thisCoin

In [53]:
[minCoins,coinUsed]=dpMakeChange1([1,5,10,21,25],63,[0]*64,[0]*64)
printCoins(coinUsed,63)
print('requires',minCoins,'coins')

21
21
21
requires 3 coins


## 动态规划
贼面前有5件宝物，分别有重量和价值，贼的背包仅能负重20共进，如何选择使得总价值最高

|item|weight|value|
|:---:|:---:|:---:|
|1|2|3|
|2|3|4|
|3|4|8|
|4|5|8|
|5|9|10|

我们记m(i,W)记为前i(1$\leq$i$\leq$5)个宝物中，组合不超过W(1$\leq$W$\leq$20)重量，得到的最大价值

$$
m(i,W)=\begin{cases}
0&\text{if}\quad i=0\\
0&\text{if}\quad W=0\\
m(i-1,W)&\text{if}\quad w_i>W\\
\max\{m(i-1,W),v_i+m(i-1,W-w_i)\}&\text{otherwise}
\end{cases}
$$

$ w_i>W$的情形就是重量太大不能继续装入宝物的情形，$i=0,w=0$代表未取得一件宝物，所以得到的价值为零

### 代码
首先定义宝物的重量和价值

In [1]:
tr=[None,{'w':2,'v':3},{'w':3,'v':4},{'w':4,'v':8},{'w':5,'v':8},{'w':9,'v':10}]
max_w=20

初始化二维表格m(i,W)

In [4]:
m={(i,w):0 for i in range(len(tr)) for w in range(max_w+1)}

逐个填写二维表格

In [5]:
for i in range(1,len(tr)):
    for w in range(1,max_w+1):
        if tr[i]['w']>w:
            m[(i,w)]=m[(i-1,w)]
        else:
            m[(i,w)]=max(m[(i-1,w)],m[(i-1,w-tr[i]['w'])]+tr[i]['v'])

输出结果

In [6]:
print(m[len(tr)-1,max_w])

29


### 递归方法

In [9]:
tr={(2,3),(3,4),(4,8),(5,8),(9,10)}

max_w=20

m={}

def thief(tr,w):
    if tr==set() or w==0:
        m[(tuple(tr),w)]=0
        return 0
    elif (tuple(tr),w) in m:
        return m[(tuple(tr),w)]
    else:
        vmax=0
        for t in tr:
            if t[0]<=w:
                v=thief(tr-{t},w-t[0])+t[1]#tr集合减去t
                vmax=max(vmax,v)
        m[(tuple(tr),w)]=vmax
        return vmax
print(thief(tr,max_w))

29


In [15]:
print(m)

{(((5, 8),), 2): 0, (((4, 8),), 1): 0, (((4, 8), (5, 8)), 6): 8, (((3, 4),), 0): 0, (((3, 4), (5, 8)), 5): 8, (((3, 4), (4, 8)), 4): 8, (((3, 4), (4, 8), (5, 8)), 9): 16, (((2, 3), (5, 8)), 4): 3, (((2, 3), (4, 8)), 3): 3, (((2, 3), (4, 8), (5, 8)), 8): 11, (((2, 3), (3, 4)), 2): 3, (((2, 3), (3, 4), (5, 8)), 7): 11, (((2, 3), (3, 4), (4, 8)), 6): 11, (((2, 3), (3, 4), (4, 8), (5, 8)), 11): 19, (((9, 10),), 6): 0, (((9, 10), (5, 8)), 11): 10, (((9, 10), (4, 8)), 10): 10, (((9, 10), (4, 8), (5, 8)), 15): 18, (((9, 10), (2, 3)), 8): 3, (((9, 10), (2, 3), (5, 8)), 13): 13, (((9, 10), (2, 3), (4, 8)), 12): 13, (((9, 10), (2, 3), (4, 8), (5, 8)), 17): 21, (((9, 10), (3, 4)), 9): 10, (((9, 10), (3, 4), (4, 8)), 13): 18, (((9, 10), (2, 3), (3, 4)), 11): 13, (((9, 10), (2, 3), (3, 4), (4, 8)), 15): 21, (((9, 10), (3, 4), (5, 8)), 14): 18, (((9, 10), (3, 4), (4, 8), (5, 8)), 18): 26, (((9, 10), (2, 3), (3, 4), (5, 8)), 16): 21, (((9, 10), (3, 4), (5, 8), (2, 3), (4, 8)), 20): 29}
